# Graphs in Cypher

This notebook gives an introduction to the Cypher query language used in the [Neo4j graph database](https://neo4j.com/) based on previous knowledge of the SQL language and the [PostgreSQL](https://www.postgresql.org/) database. See our previous notebook [`Graphs in SQL'](https://github.com/BigDataAnalyticsGroup/bigdataengineering/blob/master/Graphs%20in%20SQL.ipynb) on how to query graphs in SQL.


We use the same examples and queries as the previous notebook on SQL, however this time using just Cypher. To visualize the graphs, we use the [`vis.js`](https://visjs.org/) library as presented in the [neo4j-jupyter notebooks by Nicole White](https://github.com/nicolewhite/neo4j-jupyter).

Copyright Christian Schön & Jens Dittrich, [Big Data Analytics Group](https://bigdata.uni-saarland.de/), [CC-BY-SA](https://creativecommons.org/licenses/by-sa/4.0/legalcode)

## Setup - Neo4j

The Neo4j database which we will now set up will contain the same data as the PostgreSQL database used in first part of this notebook series, however with a slightly different layout. Instead of storing everything as tables, Neo4j distinguishes between nodes and edges.

### Neo4j

[Neo4j](https://neo4j.com/) is a graph database that is available for free for personal use. You can either [download Neo4j](https://neo4j.com/download-center/#community) and execute it locally or you can use a [blank online sandbox](https://neo4j.com/sandbox-v2/). Be careful: If you execute Neo4j locally, you have to change the password at least once and adapt the settings below before executing this notebook. In both cases (offline and online), you will need the following information to establish a connection to the database:
* ip-address and bolt port of the computer the database is running on
* username and password for authentication

If you use an online sandbox, the necessary information is displayed in the details tab after startup. A graph database does not store tables as classic RDBMS, but nodes and edges. The query language used is called [Cypher](https://neo4j.com/developer/cypher-query-language/) and differs from the SQL language of e.g. PostgreSQL.

We will first setup the same example as in PostgreSQL, i.e. using nodes labelled with `PERSON` and edges labelled with `KNOWS`, `MEETS`, or `WRITES`. In a second step, we then also implement the precedence graph example.

In [1]:
# Specify paths to CSV files for social network
contacts_persons_csv = "./data/cypher/contacts_persons.csv"
contacts_knows_csv = "./data/cypher/contacts_knows.csv"
contacts_meets_csv = "./data/cypher/contacts_meets.csv"
contacts_writes_csv = "./data/cypher/contacts_writes.csv"

# Specify paths to CSV files for precedence graph
transactions_nodes_csv = "./data/cypher/transactions_nodes.csv"
transactions_precedence_csv = "./data/cypher/transactions_precedence.csv"

In [2]:
# Import functions used to draw results in later sections
from vis.vis import drawSubgraph, print_set_postgres, print_set_neo4j
# Import the Neo4j driver and the csv package
from py2neo import Graph
import csv

# Setup connection to the Neo4j graph
# Schema of uri: "bolt://ip:port"
# Schema of auth: ("username", "password")
graph = Graph(profile="bolt://127.0.0.1:7687", auth=("neo4j", "h6u4%kr"))

# Reset graph
graph.run("MATCH (n) DETACH DELETE n;")

# Specify which property to display as label in a node
options = {
    "PERSON": "name",
    "TRANSACTION": "name"
}

# Enable physics effect
physics = True
# Set node shape: circle with labels inside or dot with labels below
node_shape = 'circle'

In [3]:
# Social network graph

# Create constraints, i.e. a primary key constraint:
graph.run("CREATE CONSTRAINT IF NOT EXISTS ON (n:Person) ASSERT n.node_id IS UNIQUE;")

# Create nodes labelled with 'PERSON'
with open(contacts_persons_csv, newline='') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        graph.run("MERGE (n:PERSON {{node_id: {}, name:'{}'}});".format(int(row['node_id']), str(row['name'])))
        
for csv_name, relationship in [(contacts_knows_csv, 'KNOWS'), (contacts_meets_csv, 'MEETS'), (contacts_writes_csv, 'WRITES')]:        
#  Create relationships labelled  'knows', 'meets' and 'writes':
    with open(csv_name, newline='') as csvfile:
        csvreader = csv.DictReader(csvfile)
        for row in csvreader:
            graph.run(("MATCH (p1:PERSON {{node_id: {}}}) MATCH (p2:PERSON {{node_id: {}}}) MERGE (p1)-[r:"+relationship+"]->(p2);").format(int(row['start_id']), int(row['end_id'])))
            

In [4]:
# Precedence graph

# Create constraints
graph.run("CREATE CONSTRAINT IF NOT EXISTS ON (n:TRANSACTION) ASSERT n.node_id IS UNIQUE;")

# Create nodes labelled with 'TRANSACTION'
with open(transactions_nodes_csv, newline='') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        graph.run("MERGE (n:TRANSACTION {{node_id: {}, name:'{}'}});".format(int(row['node_id']), str(row['name'])))

# Create relationships labelled with 'KNOWS'
with open(transactions_precedence_csv, newline='') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        graph.run("MATCH (p1:TRANSACTION {{node_id: {}}}) MATCH (p2:TRANSACTION {{node_id: {}}}) MERGE (p1)-[r:PRECEDENCE]->(p2);".format(int(row['start_id']), int(row['end_id'])))

## Social Network Graph - Neo4j

Again, our first example is the social network graph, consisting of four persons in total which might know, meet, or write with another person in the network.

### Simple Queries - Neo4j

To check if the Neo4j database is setup correctly, we use the same example as for the PostgreSQL database, i.e. querying the nodes in the database. The result should consist of four persons with IDs 0 to 4 and names 'A' to 'D'.

In [6]:
# Neo4j
# MATCH (n:PERSON) corresponds to FROM person AS n in SQL
# RETURN corresponds to SELECT in SQL
cur = graph.run("""
MATCH (n:PERSON)
RETURN n.node_id AS node_id, n.name AS name;
""")
print_set_neo4j(cur)
#cur.close()

[Result] : {[ node_id, name ]}
{
	(0, A),
	(1, B),
	(3, C),
	(2, D)
}


Which person knows which other person using directed edges:

In [7]:
cur = graph.run("""
MATCH ()-[r:KNOWS]->()
RETURN r;
""")
print_set_neo4j(cur)
#cur.close()

[Result] : {[ r ]}
{
	((A)-[:KNOWS {}]->(B)),
	((A)-[:KNOWS {}]->(C))
}


Which person knows which other person, this time using undirected edges:

In [8]:
cur = graph.run("""
MATCH ()-[r:KNOWS]-() RETURN r;
""")
print_set_neo4j(cur)
#cur.close()

[Result] : {[ r ]}
{
	((A)-[:KNOWS {}]->(C)),
	((A)-[:KNOWS {}]->(B)),
	((A)-[:KNOWS {}]->(B)),
	((A)-[:KNOWS {}]->(C))
}


We can again also output only the names of the two persons:

In [9]:
# Neo4j
# () - [...] -> () is Cypher's ASCII query syntax:
# Find all persons that know another person and return the names of both persons:
cur = graph.run("""
MATCH (n:PERSON) - [r:KNOWS] -> (n2:PERSON)
RETURN n.name, n2.name;
""")
print_set_neo4j(cur)
#cur.close()

[Result] : {[ n.name, n2.name ]}
{
	(A, B),
	(A, C)
}


## Basic Precedence Graph (Neo4j)

We now move on the basic transaction graph example already presented in the previous notebook of this series. Our task is again the search for cycles.
As a reminder, we visualize the graph first, this time by directly querying the Neo4j database.

In [10]:
# edges: (t1:TRANSACTION) --> (t2:TRANSACTION) describes a path,
# i.e. a node of type transaction leading via an edge to some other node of type transaction
# WHERE correpsonds to WHERE in SQL
# here: only consider nodes with an id <= 3.
data_neo4j = graph.run("""
MATCH edges = (a:TRANSACTION) --> (b:TRANSACTION)
WHERE a.node_id <= 3 AND b.node_id <= 3
RETURN edges;
""").to_subgraph()
drawSubgraph(data_neo4j, options, height="300", filename="basic_example.html", physics=physics, node_shape=node_shape, edge_width=3.0)

### Cypher (Naive Way)

Cypher, the query language used by Neo4j, can also detect cycles using its built-in features. However, the query remains way shorter and more readable compared to SQL. We match all transactions `a` which are related through up to 3 edges with themselves and assign the result to `cyclePath`. We then extract all nodes from `cyclePath` and store them in the variable `path`. We finally return the `node_id` of the first node on the path, the length of the path and a list containing the `node_id` of each node in the path.

In [12]:
# WITH corresponds to WITH in SQL:
# here: NODES(cyclePath) returns an ordered (index-based) list of nodes that are part of the cyclePath
# [] is a list comprehension similar to Python, however in reverse order
#  [n IN path | n.node_id] is a list of the node_ids contained in path
cur = graph.run("""
MATCH cyclePath = (a:TRANSACTION) - [*..3] -> (a)
WITH NODES(cyclePath) as path
RETURN path[0].node_id AS startID, SIZE(path) AS length, [n IN path | n.node_id] AS path;
""")
print_set_neo4j(cur)
#cur.close()

[Result] : {[ startID, length, path ]}
{
}


## Advanced Precedence Graph

In the following, we will move on to the extended example consisting of nine transactions containing multiple possible cycles as already presented above. We again visualize it using Neo4j.

In [13]:
data_neo4j = graph.run("""
MATCH edges = (a:TRANSACTION) --> (b:TRANSACTION)
RETURN edges;
""").to_subgraph()
drawSubgraph(data_neo4j, options, height="500", filename="advanced_example.html", physics=physics, node_shape=node_shape, edge_width=3.0)

### Cypher (Naive Way)

We can again apply the same query as above. The only slight adaption is the maximum length of the path which is increased to nine, the total number of nodes.

In [14]:
cur = graph.run("""
MATCH cyclePath = (a:TRANSACTION) - [*..9] -> (a)
WITH NODES(cyclePath) AS path
RETURN path[0].node_id AS startID, SIZE(path) AS length, [n IN path | n.node_id] AS path;
""")
print_set_neo4j(cur)
#cur.close()

[Result] : {[ startID, length, path ]}
{
	(1, 5, [1, 2, 6, 5, 1]),
	(1, 6, [1, 2, 9, 6, 5, 1]),
	(1, 6, [1, 3, 2, 6, 5, 1]),
	(1, 7, [1, 3, 2, 9, 6, 5, 1]),
	(1, 7, [1, 3, 8, 9, 6, 5, 1]),
	(2, 5, [2, 6, 5, 1, 2]),
	(2, 6, [2, 6, 5, 1, 3, 2]),
	(2, 6, [2, 9, 6, 5, 1, 2]),
	(2, 7, [2, 9, 6, 5, 1, 3, 2]),
	(3, 6, [3, 2, 6, 5, 1, 3]),
	(3, 7, [3, 2, 9, 6, 5, 1, 3]),
	(3, 7, [3, 8, 9, 6, 5, 1, 3]),
	(5, 5, [5, 1, 2, 6, 5]),
	(5, 6, [5, 1, 2, 9, 6, 5]),
	(5, 6, [5, 1, 3, 2, 6, 5]),
	(5, 7, [5, 1, 3, 2, 9, 6, 5]),
	(5, 7, [5, 1, 3, 8, 9, 6, 5]),
	(6, 5, [6, 5, 1, 2, 6]),
	(6, 6, [6, 5, 1, 2, 9, 6]),
	(6, 6, [6, 5, 1, 3, 2, 6]),
	(6, 7, [6, 5, 1, 3, 2, 9, 6]),
	(6, 7, [6, 5, 1, 3, 8, 9, 6]),
	(8, 7, [8, 9, 6, 5, 1, 3, 8]),
	(9, 6, [9, 6, 5, 1, 2, 9]),
	(9, 7, [9, 6, 5, 1, 3, 2, 9]),
	(9, 7, [9, 6, 5, 1, 3, 8, 9])
}


/home/vagrant/.conda/envs/bde/lib/python3.7/site-packages/py2neo/export.py:67: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(list(map(list, cursor)), dtype=dtype, order=order)


### Cypher (Using Shortest Path)

The previous query still depends on the manual adjustment of the maximal path length, but then returns all possible cycles up to this length. If we just want to know the shortest cycle per starting edge, we can adapt the query to use a shortest path algorithm which in addition removes the necessity to adapt the maximum path length.

In [15]:
cur = graph.run("""
MATCH (a:TRANSACTION)-->(b:TRANSACTION),
  cyclePath=shortestPath((b)-[*]->(a))
WITH [a] + NODES(cyclePath) AS path
RETURN path[0].node_id AS startID, SIZE(path) AS length, [n IN path | n.node_id] AS path;
""")
print_set_neo4j(cur)
#cur.close()

[Result] : {[ startID, length, path ]}
{
	(1, 6, [1, 3, 2, 6, 5, 1]),
	(1, 5, [1, 2, 6, 5, 1]),
	(2, 6, [2, 9, 6, 5, 1, 2]),
	(2, 5, [2, 6, 5, 1, 2]),
	(3, 7, [3, 8, 9, 6, 5, 1, 3]),
	(3, 6, [3, 2, 6, 5, 1, 3]),
	(5, 5, [5, 1, 2, 6, 5]),
	(6, 5, [6, 5, 1, 2, 6]),
	(8, 7, [8, 9, 6, 5, 1, 3, 8]),
	(9, 6, [9, 6, 5, 1, 2, 9])
}


## Topological Sort with Cypher

Finding a topological order for a graph essentially consists of two steps:
* Remove all cycles from the graph
* Find one possible order by recursively eliminating nodes and edges

### Remove Cycles

As we know from the previous queries that our graph does contain cycles, we first have to remove them. A naive approach could look like this: We eliminate nodes from the graph until there is no more cycle. To speed up the process, we start with the node connected to the most edges. To keep the graph intact, we do not actually remove the nodes, but mark them as removed instead.

In [16]:
# Reset the removed flag (to the value false) for all nodes with label TRANSACTION
# Return the number of affected nodes
reset_removed_flag = """
MATCH (a:TRANSACTION)
SET a.removed = false
RETURN COUNT(a) AS count;
"""

# Check the graph for cycles, i.e. if we find a path starting at node a and leading back to node a
# OPTIONAL MATCH ensures that the query returns a result even if we don't find such a path
# The WHERE clause ensures that our path does not contain any node marked as removed
# COALESCE returns the first non-NULL value and ensures that we get a boolean result 
# even if cyclePath (and therefore also LENGTH(cyclePath) is NULL)
check_graph_for_cycles = """
OPTIONAL MATCH cyclePath = (a:TRANSACTION) - [*..9] -> (a)
WHERE NOT ANY(n IN NODES(cyclePath) WHERE n.removed = true)
RETURN COALESCE(LENGTH(cyclePath) > 0, false) AS cycle;
"""

# Count the number of edges (using size((a) -- ())) connected to each node and call it degree
# Return the node ids ordered by decreasing degree
nodes_by_num_edges = """
MATCH (a:TRANSACTION)
WHERE a.removed = false
WITH a, size((a) -- ()) AS degree
RETURN a.node_id AS id
ORDER BY degree DESC, a.node_id ASC;
"""

# Mark a node as removed by setting the removed property to true
# The {} is a placeholder which will later be filled by a node_id using Python's string formatting
mark_node_as_removed = """
MATCH (a:TRANSACTION)
WHERE a.node_id = {}
SET a.removed = true
RETURN a.removed AS removed;
"""

# Retrieve the remaining graph for visulization
# First get all non-removed nodes (the part up to the WITH)
# Then use an OPTIONAL MATCH to find edges connecting them with some other non-removed nodes
retrieve_remaining_graph = """
MATCH (a:TRANSACTION)
WHERE a.removed = false
WITH a
OPTIONAL MATCH edges = (a) -- (b:TRANSACTION)
WHERE b.removed = false
RETURN a, b, edges;
"""

# First reset all removed flags
# Then loop as long as we find cycles and remove them by eliminating nodes as described above
num_nodes_resetted = graph.run(reset_removed_flag).data()[0]['count']
print("Reset removed flag for ", num_nodes_resetted, " nodes")
cycle = graph.run(check_graph_for_cycles).data()[0]['cycle']
print("Graph contains cycle? ", cycle)
while cycle:
    top_node = graph.run(nodes_by_num_edges).data()[0]['id']
    print("Node to remove next: ", top_node)
    removed = graph.run(mark_node_as_removed.format(top_node)).data()[0]['removed']
    print("Removed? ", removed)
    cycle = graph.run(check_graph_for_cycles).data()[0]['cycle']
    print("Graph contains cycle? ", cycle)

print("Remaining graph:\n")
data_neo4j = graph.run(retrieve_remaining_graph).to_subgraph()
display(drawSubgraph(data_neo4j, options, height="400", filename="remaining_graph_acyclic.html", physics=physics, node_shape=node_shape, edge_width=3.0))

Reset removed flag for  9  nodes
Graph contains cycle?  True
Node to remove next:  1
Removed?  True
Graph contains cycle?  False
Remaining graph:



### Compute Topological Order

The topological order for an acyclic, directed graph can be computed by repeatedly applying the following steps as long as the graph still contains nodes:
* Find all nodes with zero indegree, i.e. without incoming edge
* Append these nodes to the topological order
* Remove these nodes from the graph

As in the previous step, we do not remove the nodes, but only mark them as removed.

In [17]:
# Mark a node as removed, see previous section
mark_node_as_removed = """
MATCH (a:TRANSACTION)
WHERE a.node_id = {}
SET a.removed = true
RETURN a.removed AS removed;
"""

# Get all nodes with zero indegree, i.e. without an incoming edge
# First get all non-removed nodes (first part)
# Then find and count all incoming edges per node (second part)
# Finally return all nodes with zero incoming edges (third part)
nodes_zero_indegree = """
MATCH (a:TRANSACTION)
WHERE a.removed = false
WITH a
OPTIONAL MATCH (a) <-- (b:TRANSACTION)
WHERE b.removed = false
WITH a, COALESCE(COUNT(b), 0) AS size
WHERE size = 0
RETURN a.node_id AS id;
"""

# Count the number of remaining, i.e. non-removed nodes
count_remaining_nodes = """
MATCH (a:TRANSACTION)
WHERE a.removed = false
RETURN COUNT(a) AS num;
"""

# Retrieve the remaining graph for visualization, see previous section
retrieve_remaining_graph = """
MATCH (a:TRANSACTION)
WHERE a.removed = false
WITH a
OPTIONAL MATCH edges = (a) -- (b:TRANSACTION)
WHERE b.removed = false
RETURN a, b, edges;
"""

# Loop as long as we still have nodes in our graph
# Iteratively remove all nodes with zero indegree
topological_order = []
nodes_remaining = graph.run(count_remaining_nodes).data()[0]['num']
print("Nodes remaining: ", nodes_remaining)
while nodes_remaining > 0:
    data_neo4j = graph.run(retrieve_remaining_graph).to_subgraph()
    display(drawSubgraph(data_neo4j, options, height=str(nodes_remaining * 60 + 80), filename="remaining_graph_size{}.html".format(nodes_remaining), physics=physics, node_shape=node_shape, edge_width=3.0))
    next_nodes_zero_indegree = graph.run(nodes_zero_indegree).data()
    for node in next_nodes_zero_indegree:
        topological_order.append(node['id'])
        removed = graph.run(mark_node_as_removed.format(node['id'])).data()[0]['removed']
        print("Removing node ", node['id'])
    nodes_remaining = graph.run(count_remaining_nodes).data()[0]['num']
    print("\nNodes remaining: ", nodes_remaining)

print("\nFinal topological order:\n", topological_order)

Nodes remaining:  8


Removing node  3

Nodes remaining:  7


Removing node  2
Removing node  8

Nodes remaining:  5


Removing node  9

Nodes remaining:  4


Removing node  6

Nodes remaining:  3


Removing node  5

Nodes remaining:  2


Removing node  4

Nodes remaining:  1


Removing node  7

Nodes remaining:  0

Final topological order:
 [3, 2, 8, 9, 6, 5, 4, 7]


# Exercise

The following exercise will operate on the IMDb database introduced in the beginning of the lecture series. It will be loaded into Neo4j below and consists of the following node and relationship types:
* `ACTOR` nodes with attributes `node_id`, `first_name`, `last_name` and `gender`
* `DIRECTOR` nodes with attributes `node_id`, `first_name` and `last_name`
* `MOVIE` nodes with attributes `node_id`, `name`, `year` and `rank`
* `GENRE` nodes with attributes `node_id` and `name`
* `PLAYED_IN` relationship connecting actors to movies
* `DIRECTED` relationship connecting directors to movies
* `BELONGS_TO_GENRE` relationship connecting movies to genres

Some hints:
* Cypher uses the `=` operator for equality and the `<>` operator for inequality checks
* Cypher provides similar aggregation functions as the SQL standard, especially `COUNT`, `SUM` and `AVG` which might be useful in some of the exercises
* If you need to filter results based on an aggregate, use a combination of `WITH` and `WHERE` as it is shown in the `nodes_zero_indegree` query of the topological sort order section (you can ignore the `COALESCE` statement, it just ensures that 0 is returned if `COUNT(b)` returns `NULL`)
* To generate a list of nodes from a path, use the `NODES(path)` operator. Combine it with the list comprehension syntax, e.g. shown in the naive cypher query for the basic precedence graph, to get a list of specific attributes per node.
* `COUNT(DISTINCT p)` can be used to count the number of distinct paths contained in p
* If an exercise asks for paths, you can assume undirected paths by default if the exercise does not specifically ask for directed paths.
* For questions regarding the syntax, have a look at the examples above or the official [Cypher documentation](https://neo4j.com/docs/cypher-manual/current/)

## Load the IMDb example, nothing to do here

In [ ]:
# Specify paths to CSV files for social network
imdb_actors_csv = "./data/IMDb_sample/actors.csv"
imdb_directors_csv = "./data/IMDb_sample/directors.csv"
imdb_movies_csv = "./data/IMDb_sample/movies.csv"
imdb_movies_directors_csv = "./data/IMDb_sample/movies_directors.csv"
imdb_movies_genres_csv = "./data/IMDb_sample/movies_genres.csv"
imdb_roles_csv = "./data/IMDb_sample/roles.csv"

In [ ]:
from py2neo import Node, Relationship

# Reset exercise graph
graph.run("MATCH (n:ACTOR) DETACH DELETE n;")
graph.run("MATCH (n:DIRECTOR) DETACH DELETE n;")
graph.run("MATCH (n:MOVIE) DETACH DELETE n;")
graph.run("MATCH (n:GENRE) DETACH DELETE n;")

# Create constraints on node_id
graph.run("CREATE CONSTRAINT IF NOT EXISTS ON (n:ACTOR) ASSERT n.node_id IS UNIQUE;")
graph.run("CREATE CONSTRAINT IF NOT EXISTS ON (n:DIRECTOR) ASSERT n.node_id IS UNIQUE;")
graph.run("CREATE CONSTRAINT IF NOT EXISTS ON (n:MOVIE) ASSERT n.node_id IS UNIQUE;")
graph.run("CREATE CONSTRAINT IF NOT EXISTS ON (n:GENRE) ASSERT n.node_id IS UNIQUE;")

# Create nodes labelled with 'ACTOR'
print("Creating actors...")
tx = graph.begin()
actors_dict = {}
with open(imdb_actors_csv, newline='') as csvfile:
    csvreader = csv.DictReader(csvfile, delimiter="\t")
    for row in csvreader:
        actors_dict[int(row['id'])] = Node("ACTOR", node_id=int(row['id']), first_name=str(row['first_name']), last_name=str(row['last_name']), gender=str(row['gender']))
        tx.create(actors_dict[int(row['id'])])
tx.commit()
        
# Create nodes labelled with 'DIRECTOR'
print("Creating directors...")
tx = graph.begin()
directors_dict = {}
with open(imdb_directors_csv, newline='') as csvfile:
    csvreader = csv.DictReader(csvfile, delimiter="\t")
    for row in csvreader:
        directors_dict[int(row['id'])] = Node("DIRECTOR", node_id=int(row['id']), first_name=str(row['first_name']), last_name=str(row['last_name']))
        tx.create(directors_dict[int(row['id'])])
tx.commit()
        
# Create nodes labelled with 'MOVIE'
print("Creating movies...")
tx = graph.begin()
movies_dict = {}
with open(imdb_movies_csv, newline='') as csvfile:
    csvreader = csv.DictReader(csvfile, delimiter="\t")
    for row in csvreader:
        movies_dict[int(row['id'])] = Node("MOVIE", node_id=int(row['id']), name=str(row['name']), year=int(row['year']), rank=float(row['rank']))
        tx.create(movies_dict[int(row['id'])])
tx.commit()
        
# Create nodes labelled with 'GENRE' and relationships labelled with 'BELONGS_TO_GENRE'
print("Creating genres...")
tx = graph.begin()
genres = set()
genres_dict = {}
with open(imdb_movies_genres_csv, newline='') as csvfile:
    csvreader = csv.DictReader(csvfile, delimiter="\t")
    for row in csvreader:
        if not str(row['genre']) in genres:
            genres_dict[str(row['genre'])] = Node("GENRE", node_id=int(len(genres)), name=str(row['genre']))
            tx.create(genres_dict[str(row['genre'])])
            genres.add(str(row['genre']))
        tx.create(Relationship(movies_dict[int(row['movie_id'])], "BELONGS_TO_GENRE", genres_dict[str(row['genre'])]))
tx.commit()
        
# Create relationships labelled with 'PLAYED_IN'
print("Creating relationship roles...")
tx = graph.begin()
with open(imdb_roles_csv, newline='') as csvfile:
    csvreader = csv.DictReader(csvfile, delimiter="\t")
    for row in csvreader:
        tx.create(Relationship(actors_dict[int(row['actor_id'])], "PLAYED_IN", movies_dict[int(row['movie_id'])], role=str(row['role'])))
tx.commit()
        
# Create relationships labelled with 'DIRECTED'
print("Creating relationship directors...")
tx = graph.begin()
with open(imdb_movies_directors_csv, newline='') as csvfile:
    csvreader = csv.DictReader(csvfile, delimiter="\t")
    for row in csvreader:
        tx.create(Relationship(directors_dict[int(row['director_id'])], "DIRECTED", movies_dict[int(row['movie_id'])]))
tx.commit()

## Exercise 1

Compute the number of distinct movies which have a director whose name is not Quentin Tarantino. Rename the result to `num`.

In [ ]:
# write your query here
query_student_1 = """

"""

In [ ]:
# Test, do not modify
cur = graph.run(query_student_1)
data_dict_student_1 = cur.data()
    
with open("./data/cypher/tests/query1.csv", newline='') as csvfile:
    csvreader = csv.DictReader(csvfile)
    data_dict_result_1 = []
    for row in csvreader:
        data_dict_result_1.append(row)
    assert len(data_dict_result_1) == len(data_dict_student_1), "expected: " + str(len(data_dict_result_1)) + " entries, got: " + str(len(data_dict_student_1))
    for i in range(len(data_dict_result_1)):
        assert 'num' in data_dict_student_1[i], "output field 'num' missing"
        assert int(data_dict_result_1[i]['num']) == int(data_dict_student_1[i]['num']), "expected: " + str(data_dict_result_1[i]['num']) + ", got: " + str(data_dict_student_1[i]['num'])
print("Your query passed this test")

## Exercise 2

Return the average rank of all movies directed by Stanley Kubrick having more than 5 actors. Rename the output to `avgRank`.

In [ ]:
# write your query here
query_student_2 = """

"""

In [ ]:
# Test, do not modify
cur = graph.run(query_student_2)
data_dict_student_2 = cur.data()
    
with open("./data/cypher/tests/query2.csv", newline='') as csvfile:
    csvreader = csv.DictReader(csvfile)
    data_dict_result_2 = []
    for row in csvreader:
        data_dict_result_2.append(row)
    assert len(data_dict_result_2) == len(data_dict_student_2), "expected: " + str(len(data_dict_result_2)) + " entries, got: " + str(len(data_dict_student_2))
    for i in range(len(data_dict_result_2)):
        assert 'avgRank' in data_dict_student_2[i], "output field 'avgRank' missing"
        assert round(float(data_dict_result_2[i]['avgRank']), 6) == round(float(data_dict_student_2[i]['avgRank']), 6), "expected: " + str(round(float(data_dict_result_2[i]['avgRank']), 6)) + ", got: " + str(round(float(data_dict_student_2[i]['avgRank']), 6))
print("Your query passed this test")

## Exercise 3

Compute the number of distinct paths of minimal length 2 and maximal length 5 between the two actors Sally Ricca and Martin Jarvis. Rename the output to `numPaths`.

In [ ]:
# write your query here
query_student_3 = """

"""

In [ ]:
# Test, do not modify
cur = graph.run(query_student_3)
data_dict_student_3 = cur.data()
    
with open("./data/cypher/tests/query5.csv", newline='') as csvfile:
    csvreader = csv.DictReader(csvfile)
    data_dict_result_3 = []
    for row in csvreader:
        data_dict_result_3.append(row)
    assert len(data_dict_result_3) == len(data_dict_student_3), "expected: " + str(len(data_dict_result_3)) + " entries, got: " + str(len(data_dict_student_3))
    for i in range(len(data_dict_result_3)):
        assert 'numPaths' in data_dict_student_3[i], "output field 'numPaths' missing"
        assert int(data_dict_result_3[i]['numPaths']) == int(data_dict_student_3[i]['numPaths']), "expected: " + str(data_dict_result_3[i]['numPaths']) + ", got: " + str(data_dict_student_3[i]['numPaths'])
print("Your query passed this test")

## Exercise 4

Compute the shortest path between the two actors Ken Gibbel and Vince Pace. Return the path as a list of `node_id` (the id of each node contained in the path) and rename it to `shortestPath`.

In [ ]:
# write your query here
query_student_4 = """

"""

In [ ]:
# Test, do not modify
cur = graph.run(query_student_4)
data_dict_student_4 = cur.data()

with open("./data/cypher/tests/query6.csv", newline='') as csvfile:
    csvreader = csv.DictReader(csvfile, delimiter='\t')
    data_dict_result_4 = []
    for row in csvreader:
        data_dict_result_4.append(row)
    assert len(data_dict_result_4) == len(data_dict_student_4), "expected: " + str(len(data_dict_result_4)) + " entries, got: " + str(len(data_dict_student_4))
    for i in range(len(data_dict_result_4)):
        assert 'shortestPath' in data_dict_student_4[i], "output field 'shortestPath' missing"
        assert str(data_dict_result_4[i]['shortestPath']) == str(data_dict_student_4[i]['shortestPath']), "expected: " + str(data_dict_result_4[i]['shortestPath']) + ", got: " + str(data_dict_student_4[i]['shortestPath'])
print("Your query passed this test")